In [1]:
import numpy as np
import pandas as pd
import csv
from dask.distributed import Client, progress
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

In [2]:
client = Client(processes=False, threads_per_worker=4,
                n_workers=4, memory_limit='8GB')

In [3]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.0.105:8787/status,
Dashboard: http://192.168.0.105:8787/status,Workers: 4
Total threads: 16,Total memory: 29.80 GiB
Status: running,Using processes: False
Comm: inproc://192.168.0.105/32764/1,Workers: 4
Dashboard: http://192.168.0.105:8787/status,Total threads: 16
Started: Just now,Total memory: 29.80 GiB
Comm: inproc://192.168.0.105/32764/4,Total threads: 4
Dashboard: http://192.168.0.105:51823/status,Memory: 7.45 GiB
Nanny: None,


In [4]:
df = dd.read_csv('data/train2.csv', dtype={"district_id": "Int64", "city_id": "Int64", "id": "Int64", "is_business": "Int64", "region_id": "Int64", "price": "Int64", "created_at_first": "str",
"params": "str"}, engine="python", encoding="utf-8")

In [5]:
df.head()

,Unnamed: 0,id,created_at_first,category,is_business,district_id,city_id,region_id,params,price
0,0,325017,2018-10-06 12:20:48,Mieszkanie na sprzedaż,1,12867,210,14,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556
1,1,513427,2019-02-18 12:17:51,Mieszkanie na sprzedaż,1,<NA>,3614,15,price[currency]<=>PLN<br>m<=>95<br>rooms_num<=...,500000
2,2,824979,2019-03-21 17:31:09,Mieszkanie na sprzedaż,1,3434,204,10,price[currency]<=>PLN<br>m<=>58.9<br>rooms_num...,238000
3,3,400727,2018-11-28 23:50:45,Mieszkanie na sprzedaż,1,87,1,15,price[currency]<=>PLN<br>m<=>45<br>rooms_num<=...,209000
4,4,298324,2019-01-22 21:22:42,Dom na sprzedaż,0,<NA>,11879,7,price[currency]<=>PLN<br>terrain_area<=>3201<b...,729000


In [6]:
df.isnull().sum().compute()

Unnamed: 0               0
id                       0
created_at_first         0
category                 0
is_business              0
district_id         294251
city_id                  0
region_id                0
params                   0
price                    0
dtype: int64

In [6]:
#df["district_id"] = df["district_id"].astype('Int64')

In [5]:
districts = dd.read_csv("data/districts.csv", dtype={'id': 'Int64'})

In [8]:
districts.head()

,id,name_pl,lon,lat
0,1,Stare Miasto,17.01434,51.10874
1,2,Śródmieście,17.08127,51.11324
2,3,Krzyki,17.05319,51.07855
3,4,Psie Pole,17.03559,51.14586
4,5,Fabryczna,16.98647,51.11228


In [6]:
cities = dd.read_csv("data/cities.csv", dtype={'id': 'Int64'})

In [10]:
cities.head()

,id,name_pl,lon,lat
0,423,Dębno,20.71895,49.95904
1,498,Lipnica Wielka,19.62836,49.48472
2,679,Grudusk,20.62545,53.05881
3,828,Wyszogród,20.19201,52.38748
4,1068,Regnów,20.39088,51.74762


In [7]:
data = dd.merge(df, cities, left_on="city_id", right_on="id")

In [12]:
data.tail(20)

,Unnamed: 0,id_x,created_at_first,category,is_business,district_id,city_id,region_id,params,price,id_y,name_pl,lon,lat
119022,831497,96557,2018-02-08 21:52:48,Dom na sprzedaż,1,<NA>,62196,8,price[currency]<=>PLN<br>terrain_area<=>1263<b...,550000,62196,Szczepanek,18.34143,50.53281
119023,831508,503958,2018-02-27 22:43:48,Dom na sprzedaż,1,<NA>,819,7,price[currency]<=>PLN<br>terrain_area<=>70400<...,520000,819,Gąbin,19.73374,52.39930
119024,831519,327717,2018-10-18 15:31:52,Dom na sprzedaż,1,<NA>,16406,4,price[currency]<=>PLN<br>terrain_area<=>1060<b...,360000,16406,Sieniawa Żarska,15.06519,51.63974
119025,831579,135913,2018-04-21 11:22:35,Mieszkanie na sprzedaż,0,<NA>,270,15,price[currency]<=>PLN<br>m<=>67<br>rooms_num<=...,243000,270,Opatówek,18.22257,51.73967
119026,831602,12644,2018-07-21 22:48:05,Dom na sprzedaż,0,<NA>,867,7,price[currency]<=>PLN<br>terrain_area<=>1600<b...,240000,867,Gózd,21.38918,51.37486
119027,831650,948671,2018-06-13 15:11:18,Dom na sprzedaż,1,<NA>,28204,8,price[currency]<=>PLN<br>terrain_area<=><br>m<...,289999,28204,Bodzanowice,18.62771,50.91181
119028,831675,385001,2018-09-08 08:51:06,Dom na sprzedaż,0,<NA>,17606,15,price[currency]<=>PLN<br>terrain_area<=>751<br...,268000,17606,Radzyny,16.58330,52.54227
119029,831748,575320,2018-05-09 17:07:39,Dom na sprzedaż,1,<NA>,5932,6,price[currency]<=>PLN<br>terrain_area<=>300<br...,235000,5932,Brodła,19.59000,50.04243
119030,831760,186863,2018-04-26 12:57:05,Dom na sprzedaż,1,<NA>,35800,6,price[currency]<=>PLN<br>terrain_area<=>20000<...,830000,35800,Żegiestów,20.80003,49.37670
119031,831789,807736,2018-10-21 23:11:01,Dom na sprzedaż,1,<NA>,11712,6,price[currency]<=>PLN<br>terrain_area<=>400<br...,420000,11712,Kochanów,19.75058,50.11429


In [8]:
data = data.drop("id_y", axis=1)

In [13]:
#districts["id"] = districts["id"].astype('Int64')

In [9]:
districts = districts.rename(columns={"id": "district_id"})

In [15]:
districts.head()

,district_id,name_pl,lon,lat
0,1,Stare Miasto,17.01434,51.10874
1,2,Śródmieście,17.08127,51.11324
2,3,Krzyki,17.05319,51.07855
3,4,Psie Pole,17.03559,51.14586
4,5,Fabryczna,16.98647,51.11228


In [10]:
data2 = dd.merge(data, districts, on="district_id", how="left")

In [17]:
data2.tail()

,Unnamed: 0,id_x,created_at_first,category,is_business,district_id,city_id,region_id,params,price,name_pl_x,lon_x,lat_x,name_pl_y,lon_y,lat_y
119037,831877,885732,2018-06-13 11:01:23,Dom na sprzedaż,1,<NA>,1196,1,price[currency]<=>PLN<br>terrain_area<=>1900<b...,309000,Jemielno,16.51654,51.56101,NaN,NaN,NaN
119038,831888,377944,2018-10-24 13:06:18,Mieszkanie na sprzedaż,1,<NA>,26942,1,price[currency]<=>PLN<br>m<=>96<br>rooms_num<=...,139000,Goszczyna,17.19292,50.86717,NaN,NaN,NaN
119039,831931,166447,2018-06-25 22:32:34,Dom na sprzedaż,0,<NA>,43962,11,price[currency]<=>PLN<br>terrain_area<=>11500<...,1030000,Miszewo,17.06295,54.26565,NaN,NaN,NaN
119040,831938,598241,2018-09-04 13:00:27,Dom na sprzedaż,1,<NA>,10511,12,price[currency]<=>PLN<br>terrain_area<=>1621<b...,1200000,Stanica,18.51538,50.20051,NaN,NaN,NaN
119041,831946,310611,2018-02-07 19:23:44,Dom na sprzedaż,1,<NA>,18681,15,price[currency]<=>PLN<br>terrain_area<=>1352<b...,360000,Rostworowo,16.76282,52.53778,NaN,NaN,NaN


In [18]:
data2.head()

,Unnamed: 0,id_x,created_at_first,category,is_business,district_id,city_id,region_id,params,price,name_pl_x,lon_x,lat_x,name_pl_y,lon_y,lat_y
0,0,325017,2018-10-06 12:20:48,Mieszkanie na sprzedaż,1,12867,210,14,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556,Olsztyn,20.47703,53.77602,Dajtki,20.42342,53.76516
1,107,376299,2018-07-03 10:20:56,Mieszkanie na sprzedaż,1,280,210,14,price[currency]<=>PLN<br>m<=>49.6<br>rooms_num...,222000,Olsztyn,20.47703,53.77602,Kormoran,20.49830,53.76785
2,145,152946,2018-12-05 13:36:08,Mieszkanie na sprzedaż,1,590,210,14,price[currency]<=>PLN<br>m<=>39.41<br>rooms_nu...,254195,Olsztyn,20.47703,53.77602,Nagórki,20.49327,53.75620
3,249,266877,2018-10-16 18:00:31,Dom na sprzedaż,1,2568,210,14,price[currency]<=>PLN<br>terrain_area<=>703<br...,690000,Olsztyn,20.47703,53.77602,Redykajny,20.43742,53.81064
4,351,710096,2018-12-07 12:26:47,Mieszkanie na sprzedaż,1,6168,210,14,price[currency]<=>PLN<br>m<=>36.53<br>rooms_nu...,209000,Olsztyn,20.47703,53.77602,Osiedle Mazurskie,20.51410,53.76086


In [11]:
data = data2.drop(columns = {"Unnamed: 0", "id_x", "created_at_first", "region_id", "lon_x", "lat_x"})

In [16]:
data.head()

,category,is_business,district_id,city_id,params,price,name_pl_x,name_pl_y,name_pl_y,lon_y,lat_y
0,Mieszkanie na sprzedaż,1,12867,210,price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...,389556,Olsztyn,Dajtki,Dajtki,20.42342,53.76516
1,Mieszkanie na sprzedaż,1,280,210,price[currency]<=>PLN<br>m<=>49.6<br>rooms_num...,222000,Olsztyn,Kormoran,Kormoran,20.49830,53.76785
2,Mieszkanie na sprzedaż,1,590,210,price[currency]<=>PLN<br>m<=>39.41<br>rooms_nu...,254195,Olsztyn,Nagórki,Nagórki,20.49327,53.75620
3,Dom na sprzedaż,1,2568,210,price[currency]<=>PLN<br>terrain_area<=>703<br...,690000,Olsztyn,Redykajny,Redykajny,20.43742,53.81064
4,Mieszkanie na sprzedaż,1,6168,210,price[currency]<=>PLN<br>m<=>36.53<br>rooms_nu...,209000,Olsztyn,Osiedle Mazurskie,Osiedle Mazurskie,20.51410,53.76086


In [28]:
data = data.rename(columns = {"name_pl_x": "city_name", "name_pl_y": "disctrict_name"})

In [57]:
data["params"].head(1)

0    price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...
Name: params, dtype: object

0         price[currency]<=>PLN<br>m<=>72.14<br>rooms_nu...
1         price[currency]<=>PLN<br>m<=>95<br>rooms_num<=...
2         price[currency]<=>PLN<br>m<=>58.9<br>rooms_num...
3         price[currency]<=>PLN<br>m<=>45<br>rooms_num<=...
4         price[currency]<=>PLN<br>terrain_area<=>3201<b...
                                ...                        
119037    price[currency]<=>PLN<br>m<=>39.45<br>rooms_nu...
119038    price[currency]<=>PLN<br>m<=>48<br>rooms_num<=...
119039    price[currency]<=>PLN<br>m<=>56.42<br>rooms_nu...
119040    price[currency]<=>PLN<br>m<=>51.85<br>rooms_nu...
119041    price[currency]<=>PLN<br>m<=>60.08<br>rooms_nu...
Name: params, Length: 832011, dtype: object

In [42]:
len(data2["district_id"])

832011